In [1]:
import time
import math

import numpy as np
import pandas as pd
from itertools import product
from services.big_m_strategies import *
from services.experiments import *
from services.plotting_functions import *

In [2]:
# Some strategies (uncomment this or the next cell to pick a strategy to test)
# Strategy = general_strategy(sample_estimator, MVO, NumObs=48)  #the general strategy object takes an estimator function and an optimizer function
# the only constraint is that the estimators outputs are the optimizers inputs
# investor preferences (static throughout the backtest

universe = 'SP'
initialVal = 100000
estimator_str = 'exponential_weighted_estimator_shrinkage'
premium_strs = ['0.25']
kappas_SP = ['0.0125', '0.025', '0.05']
kappas_ETF = ['0.4', '0.8', '1.6']
cardinality_strs_SP = ['0.02', '0.03', '0.05', '0.07', '0.1', '0.2', '0.4', '0.6', '0.8']
cardinality_strs_ETF = ['0.01', '0.03', '0.05', '0.07', '0.1', '0.2', '0.4', '0.6', '0.8']
q = 20

In [3]:
CardMVO_SP_path = "Experiments//SP//"+"CardMVO"
CardMVO_ETF_path = "Experiments//ETF//"+"CardMVO"

CardMVO_SP_experiments = pd.read_pickle(CardMVO_SP_path+"//data_dictionary.pkl")
CardMVO_ETF_experiments = pd.read_pickle(CardMVO_ETF_path+"//data_dictionary.pkl")

CardMVO_SP_experiments.run.unique() # select run

array(['SunJan212024'], dtype=object)

In [4]:
# Filtering step for CardMVO experiments
CardMVO_SP_experiments = CardMVO_SP_experiments[CardMVO_SP_experiments.estimator == estimator_str]
CardMVO_SP_experiments = CardMVO_SP_experiments[CardMVO_SP_experiments.q == str(q)]
CardMVO_SP_experiments = CardMVO_SP_experiments[CardMVO_SP_experiments.premium.isin(premium_strs)]
CardMVO_SP_experiments = CardMVO_SP_experiments[CardMVO_SP_experiments.cardinality_ratio.isin(cardinality_strs_SP)]

#CardMVO_SP_experiments = CardMVO_SP_experiments[CardMVO_SP_experiments.kappa.isin(kappas_SP)]

CardMVO_ETF_experiments = CardMVO_ETF_experiments[CardMVO_ETF_experiments.estimator == estimator_str]
CardMVO_ETF_experiments = CardMVO_ETF_experiments[CardMVO_ETF_experiments.q == str(q)]
CardMVO_ETF_experiments = CardMVO_ETF_experiments[CardMVO_ETF_experiments.premium.isin(premium_strs)]
#CardMVO_ETF_experiments = CardMVO_ETF_experiments[CardMVO_ETF_experiments.kappa.isin(kappas_ETF)]
CardMVO_ETF_experiments = CardMVO_ETF_experiments[CardMVO_ETF_experiments.cardinality_ratio.isin(cardinality_strs_ETF)]


In [5]:
SP_unique_card_turnover_df = CardMVO_SP_experiments.groupby(['cardinality_ratio', 'turnover_limit', 'premium']).uid.first() # unique experiments
ETF_unique_card_turnover_df = CardMVO_ETF_experiments.groupby(['cardinality_ratio', 'turnover_limit', 'premium']).uid.first() # unique experiments

In [6]:
CardMVO_ETF_experiments.premium.unique()

array(['0.25'], dtype=object)

In [7]:
# gap and time stats
card_mvo_gaps = []
card_mvo_times = []

for i in SP_unique_card_turnover_df.index:
    Card_info = import_dict(CardMVO_SP_path, SP_unique_card_turnover_df[i], 'backtest_results' )
    t_s = Card_info.keys()
    cardinality_ratio = i[0]
    if float(cardinality_ratio) <= 0.2:
        for t in t_s:    
            card_mvo_gaps.append(Card_info[t]['optimality gap'])
            card_mvo_times.append(Card_info[t]['time'])

for i in ETF_unique_card_turnover_df.index:
    Card_info = import_dict(CardMVO_ETF_path, ETF_unique_card_turnover_df[i], 'backtest_results' )
    t_s = Card_info.keys()
    cardinality_ratio = i[0]
    if float(cardinality_ratio) <= 0.2:
        for t in t_s:    
            card_mvo_gaps.append(Card_info[t]['optimality gap'])
            card_mvo_times.append(Card_info[t]['time'])


In [11]:
Card_info[0].keys()

dict_keys(['obj_value', 'time', 'optimality gap', 'x', 'z', 'feasible_solution', 'optimization_params', 'calEnd'])

In [12]:
print("number of instances ", len(card_mvo_gaps))
print("average Card-MVO optimality gap ", np.mean(card_mvo_gaps))
print("std deviation Card-MVO optimality gap ", np.std(card_mvo_gaps))
print("average Card-MVO time", np.mean(card_mvo_times) )
print("number of instances with above 5%", np.sum(np.array(card_mvo_gaps)>5) )

number of instances  1386
average Card-MVO optimality gap  2.7079812334934927
std deviation Card-MVO optimality gap  3.434043798913321
average Card-MVO time 2.0302548733624546
number of instances with above 5% 78


In [13]:
# read in SVMMVO experiments and see what is going on
SVMMVO_SP_path = "Experiments//SP//"+"ClassWgtSVMMVO"+"//"+str(q)
SVMMVO_ETF_path = "Experiments//ETF//"+"ClassWgtSVMMVO"+"//"+str(q)

SVMMVO_SP_experiments = pd.read_pickle(SVMMVO_SP_path+"//data_dictionary.pkl")
SVMMVO_ETF_experiments = pd.read_pickle(SVMMVO_ETF_path+"//data_dictionary.pkl")

print("q =", q, " runs available: ", SVMMVO_SP_experiments.run.unique())

q = 20  runs available:  ['SunJan212024']


In [14]:
# filtering step
SVMMVO_SP_experiments =SVMMVO_SP_experiments[SVMMVO_SP_experiments.estimator == estimator_str]
SVMMVO_SP_experiments =SVMMVO_SP_experiments[SVMMVO_SP_experiments.premium.isin(premium_strs)]
SVMMVO_SP_experiments =SVMMVO_SP_experiments[SVMMVO_SP_experiments.kappa.isin(kappas_SP)]
SVMMVO_SP_experiments =SVMMVO_SP_experiments[SVMMVO_SP_experiments.cardinality_ratio.isin(cardinality_strs_SP)]

SVMMVO_ETF_experiments =SVMMVO_ETF_experiments[SVMMVO_ETF_experiments.estimator == estimator_str]
SVMMVO_ETF_experiments =SVMMVO_ETF_experiments[SVMMVO_ETF_experiments.premium.isin(premium_strs)]
SVMMVO_ETF_experiments =SVMMVO_ETF_experiments[SVMMVO_ETF_experiments.kappa.isin(kappas_ETF)]
SVMMVO_ETF_experiments =SVMMVO_ETF_experiments[SVMMVO_ETF_experiments.cardinality_ratio.isin(cardinality_strs_ETF)]

In [31]:
# prepare the results dictionary for
SP_unique_svmmvo_turnover_df = SVMMVO_SP_experiments.groupby(['cardinality_ratio', 'turnover_limit', 'kappa', 'premium']).uid.first()
ETF_unique_svmmvo_turnover_df = SVMMVO_ETF_experiments.groupby(['cardinality_ratio', 'turnover_limit', 'kappa', 'premium']).uid.first()

SVMMVO_bigm_times = []
SVMMVO_times = []
SVMMVO_gaps = []
for i in SP_unique_svmmvo_turnover_df.index:
    SVMMVO_information = import_dict(SVMMVO_SP_path, SP_unique_svmmvo_turnover_df[i], 'backtest_results' )
    t_s = SVMMVO_information.keys()
    for t in t_s:
        SVMMVO_bigm_times.append(SVMMVO_information[t]['bigM_time'])
        SVMMVO_gaps.append(SVMMVO_information[t]['optimality gap'])
        SVMMVO_times.append(SVMMVO_information[t]['time'])
        
for i in ETF_unique_svmmvo_turnover_df.index:
    SVMMVO_information = import_dict(SVMMVO_ETF_path, ETF_unique_svmmvo_turnover_df[i], 'backtest_results' )
    t_s = SVMMVO_information.keys()
    for t in t_s:
        SVMMVO_bigm_times.append(SVMMVO_information[t]['bigM_time'])
        SVMMVO_gaps.append(SVMMVO_information[t]['optimality gap'])
        SVMMVO_times.append(SVMMVO_information[t]['time'])

In [32]:
print("number of instances ", len(SVMMVO_bigm_times))
print("average SVMMVO optimality gap ", np.mean(SVMMVO_gaps))
print("std deviation SVMMVO optimality gap ", np.std(SVMMVO_gaps))
print("average SVMMVO time", np.mean(SVMMVO_times) )
print("average big M SVMMVO time", np.mean(SVMMVO_bigm_times) )
print("number of instances with above 5%", np.sum(np.array(SVMMVO_gaps)>5) )

number of instances  6237
average SVMMVO optimality gap  13.695655679387896
std deviation SVMMVO optimality gap  13.593782461066366
average SVMMVO time 66.15359854549098
average big M SVMMVO time 53.892955280409154
number of instances with above 5% 3064


In [36]:
SVMMVO_information[0].keys()

dict_keys(['obj_value', 'time', 'bigM_time', 'optimality gap', 'x', 'z', 'w', 't', 'b', 'xi_plus', 'xi_neg', 'feasible_solution', 'C', 'epsilon', 'q', 'optimization_params', 'calEnd'])

In [38]:
np.sqrt(min(2*SVMMVO_information[0]['obj_value']/SVMMVO_information[0]['epsilon'], 2*SVMMVO_information[0]['C']))

18.841275669597675

In [35]:
np.max(SVMMVO_gaps)

61.53846133981342